# Low-latency item-to-item recommendation system 

## Part 1 - Creating ANN index

## Overview

This notebook is a part of the [**Low-latency item-to-item recommendation system** ML Engineering blueprint](https://github.com/jarokaz/analytics-componentized-patterns/tree/master/retail/recommendation-system/bqml-ann).

The blueprint provides guidance and code samples for how to develop and operationalize a near real-time item-to-itme recommendations system that utilizes BigQuery, BigQuery ML and AI Platform ANN Service or the open source ScaNN library.

The notebook demonstrates how to create and deploy an ANN index using embeddings created in the preceding notebooks. In this notebook you go through the following steps.

1. Exporting embeddings from BigQuery into the JSONL formated file.
2. Creating an ANN Index using the exported embeddings.
3. Creating and ANN Endpoint. 
4. Deploying the ANN Index to the ANN Endpoint.
5. Testing the deployed ANN Index.

This notebook was designed to run on [AI Platform Notebooks](https://cloud.google.com/ai-platform/notebooks/docs) using the standard TensorFlow 2.4 image. Your notebook instance should be in the same project as the AI Platform ANN Service.

While AI Platform ANN Service is in the Experimental stage your project must be allow-listed before using the service. Contact `gcp-ann-feedback@` to allow list your project and user id.

## Setting up the notebook's environment

### Configuring AI Platform ANN Service

#### Enable the required Cloud APIs

You need to enable the following APIs to use the ANN service:
* aiplatform.googleapis.com
* servicenetworking.googleapis.com
* compute.googleapis.com

#### Prepare a VPC network

In the experimental release, ANN service is only accessible using private endpoints. Before using the service you need to have a [VPC network](https://cloud.google.com/vpc) configured with [private services access](https://cloud.google.com/vpc/docs/configure-private-services-access). You can use the `default` VPC or create a new one.

The below instructions are for a VPC that was created with auto subnets and regional dynamic routing mode (defaults). It is recommended that you execute the below commands from Cloud Shell, using the account with appropriate permissions - `roles/compute.networkAdmin`.

1. Set environment variables for your project ID, the name of your VPC network, and the name of your reserved range of addresses. The name of the reserved range can be an arbitrary name. It is for display only.

```
PROJECT_ID=<your-project-id>
gcloud config set project $PROJECT_ID
NETWORK_NAME=<your-VPC-network-name>
PEERING_RANGE_NAME=google-reserved-range

```

2. Reserve an IP range for Google services. The reserved range should be large enought to accommodate all peered services. The below command reserves a CIDR block with mask /16

```
gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service: AI Platform Online Prediction" \
  --network=$NETWORK_NAME \
  --purpose=VPC_PEERING \
  --project=$PROJECT_ID

```

3. Create a private connection to establish a VPC Network Peering between your VPC network and the Google services network.

```
gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK_NAME \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

```


### Importing notebook dependencies

In [10]:
import base64
import datetime
import logging
import os
import json
import pandas as pd
import time
import sys

import grpc

import google.auth
import numpy as np
import tensorflow.io as tf_io

from google.cloud import bigquery
from typing import List, Optional, Text, Tuple

In the experimental release, the deployed ANN index is exposed throught the GRPC interface. The `ann_grpc` folder contains the grpc client stub to interface to the service.

In [11]:
ANN_GRPC_ENDPOINT_STUB = 'ann_grpc'
if ANN_GRPC_ENDPOINT_STUB not in sys.path:
    sys.path.append(ANN_GRPC_ENDPOINT_STUB)

In [12]:
import ann_grpc.match_pb2_grpc as match_pb2_grpc
import ann_grpc.match_pb2 as match_pb2

### Configure GCP environment

Set the following constants to the values reflecting your environment:

* `PROJECT_ID` - your GCP project ID.
* `PROJECT_NUMBER` - your GCP project number.
* `BQ_DATASET_NAME` - the name of the BigQuery dataset that contains the item embeddings table.
* `BQ_LOCATION` - the dataset location
* `DATA_LOCATION` - a GCS location for the exported embeddings (JSONL) files.
* `VPC_NAME` - a name of the GCP VPC to use for the index deployments. Use the name of the VPC prepared in the previous section. 
* `REGION` - a compute region. Don't change the default - `us-central` - while the ANN Service is in the experimental stage


In [14]:
PROJECT_ID = 'jk-mlops-dev' # <-CHANGE THIS
PROJECT_NUMBER = '895222332033' # <-CHANGE THIS
BQ_DATASET_NAME = 'song_embeddings' # <- CHANGE THIS
BQ_LOCATION = 'US' # <- CHANGE THIS
DATA_LOCATION = 'gs://jk-ann-staging/embeddings' # <-CHANGE THIS
VPC_NAME = 'default' # <-CHANGE THIS

If you did not customize the preceding notebooks you don't need to modify the following constants.

In [15]:
EMBEDDINGS_TABLE = 'item_embeddings'
REGION = 'us-central1'
MATCH_SERVICE_PORT = 10000

## Exporting the embeddings

In the preceeding notebookes you trained the Matrix Factorization BQML model and exported the embeddings to the `item_embeddings` table. 

In this step you will extract the embeddings to a set of JSONL files in the format required by the ANN service.

### Verify the number of embeddings

In [16]:
client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

In [17]:
query = f"""
    SELECT COUNT(*) embedding_count
    FROM {BQ_DATASET_NAME}.item_embeddings;
"""

query_job = client.query(query)
query_job.to_dataframe()

,embedding_count
0,35519


### Export the embeddings

In [20]:
file_name_pattern = 'embedding-*.json'
destination_uri = f'{DATA_LOCATION}/{file_name_pattern}'
table_id = 'item_embeddings'
destination_format = 'NEWLINE_DELIMITED_JSON'

dataset_ref = bigquery.DatasetReference(PROJECT_ID, BQ_DATASET_NAME)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

extract_job = client.extract_table(
    table_ref,
    destination_uris=destination_uri,
    job_config=job_config,
    #location=BQ_LOCATION,
)  

extract_job.result()

Inspect the extracted files.

In [21]:
! gsutil ls {DATA_LOCATION}

gs://jk-ann-staging/embeddings/embedding-000000000000.json


## Creating an ANN index deployment

Deploying an ANN index is a 3 step process:
1. Creating/updating an index from source files
2. Creating an endpoint to access the index
3. Deploying the index to the endpoint


You will use the REST interface to invoke the AI Platform ANN Service Control Plane API that manages indexes, endpoints, and deployments.

After the index has been deployed you can submit matching requests using Online Querying API. In the experimental stage this API is only accessible through the gRPC interface.

Refer to [ANN service user guide](https://docs.google.com/document/d/1D5HsS829UuYUuyCTHWSFqR8dl4KNvL9C82nEs9vLtjg/edit) for more information about the ANN Service APIs.

### Define helper classes to encapsulate the ANN Service REST API.

Currently, there is no Python client that encapsulates the ANN Service Control Plane API. The below code snippet defines a simple wrapper that encapsulates a subset of REST APIs used in this notebook.



In [38]:
class ANNClient(object):
    """Base ANN Service client."""
    
    def __init__(self, project_id, project_number, region):
        credentials, _ = google.auth.default()
        self.authed_session = google.auth.transport.requests.AuthorizedSession(credentials)
        self.ann_endpoint = f'{region}-aiplatform.googleapis.com'
        self.ann_parent = f'https://{self.ann_endpoint}/v1alpha1/projects/{project_id}/locations/{region}'
        self.project_id = project_id
        self.project_number = project_number
        self.region = region
        
    def wait_for_completion(self, operation_id, message, sleep_time):
        """Waits for a completion of a long running operation."""
        
        api_url = f'{self.ann_parent}/operations/{operation_id}'

        start_time = datetime.datetime.utcnow()
        while True:
            response = self.authed_session.get(api_url)
            if 'done' in response.json().keys():
                logging.info('Operation completed!')
                break
            elapsed_time = datetime.datetime.utcnow() - start_time
            logging.info('{}. Elapsed time since start: {}.'.format(
                message, str(elapsed_time)))
            time.sleep(sleep_time)
    
        return response.json()['response']


class IndexClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN indexes."""

    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_index(self, display_name, description, metadata):
        """Creates an ANN Index."""
    
        api_url = f'{self.ann_parent}/indexes'
    
        request_body = {
            'display_name': display_name,
            'description': description,
            'metadata': metadata
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id

    def list_indexes(self, display_name=None):
        """Lists all indexes with a given display name or
        all indexes if the display_name is not provided."""
    
        if display_name:
            api_url = f'{self.ann_parent}/indexes?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexes'

        response = self.authed_session.get(api_url).json()

        return response['indexes'] if response else []
    
    def delete_index(self, index_id):
        """Deletes an ANN index."""
        
        api_url = f'{self.ann_parent}/indexes/{index_id}'
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)


class IndexDeploymentClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN endpoints and deployments."""
    
    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_endpoint(self, display_name, vpc_name):
        """Creates an ANN endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints'
        network_name = f'projects/{self.project_number}/global/networks/{vpc_name}'

        request_body = {
            'display_name': display_name,
            'network': network_name
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
    
        return operation_id
    
    def list_endpoints(self, display_name=None):
        """Lists all ANN endpoints with a given display name or
        all endpoints in the project if the display_name is not provided."""
        
        if display_name:
            api_url = f'{self.ann_parent}/indexEndpoints?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexEndpoints'

        response = self.authed_session.get(api_url).json()
 
        return response['indexEndpoints'] if response else []
    
    def delete_endpoint(self, endpoint_id):
        """Deletes an ANN endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'
        
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response.json()
    
    def create_deployment(self, display_name, deployment_id, endpoint_id, index_id):
        """Deploys an ANN index to an endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:deployIndex'
        index_name = f'projects/{self.project_number}/locations/{self.region}/indexes/{index_id}'

        request_body = {
            'deployed_index': {
                'id': deployment_id,
                'index': index_name,
                'display_name': display_name
            }
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id
    
    def get_deployment_grpc_ip(self, endpoint_id, deployment_id):
        """Returns a private IP address for a gRPC interface to 
        an Index deployment."""
  
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'

        response = self.authed_session.get(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
            
        endpoint_ip = None
        if 'deployedIndexes' in response.json().keys():
            for deployment in response.json()['deployedIndexes']:
                if deployment['id'] == deployment_id:
                    endpoint_ip = deployment['privateEndpoints']['matchGrpcAddress']
                    
        return endpoint_ip

    
    def delete_deployment(self, endpoint_id, deployment_id):
        """Undeployes an index from an endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:undeployIndex'
        
        request_body = {
            'deployed_index_id': deployment_id
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response
    
    
index_client = IndexClient(PROJECT_ID, PROJECT_NUMBER, REGION)
deployment_client = IndexDeploymentClient(PROJECT_ID, PROJECT_NUMBER, REGION)

### Create an index

#### List all indexes registered with the ANN service

In [33]:
indexes = index_client.list_indexes()

if not indexes:
    print('There are not any indexes registered with the service')

for index in indexes:
    print(index['name'])

There are not any indexes registered with the service


#### Configure and create a new index based on the exported embeddings

Index creation is a long running operation. Be patient.

In [34]:
index_display_name = 'Song embeddings'
index_description = 'Song embeddings created BQML Matrix Factorization model'

index_metadata = {
    'contents_delta_uri': DATA_LOCATION,
    'config': {
        'dimensions': 50,
        'approximate_neighbors_count': 50,
        'distance_measure_type': 'DOT_PRODUCT_DISTANCE',
        'feature_norm_type': 'UNIT_L2_NORM',
        'tree_ah_config': {
            'child_node_count': 1000,
            'max_leaves_to_search': 100
         }
    }
}

In [35]:
logging.getLogger().setLevel(logging.INFO)

operation_id = index_client.create_index(index_display_name, 
                                          index_description,
                                          index_metadata)

response = index_client.wait_for_completion(operation_id, 'Creating index', 20)
print(response)

INFO:root:Creating index. Elapsed time since start: 0:00:00.097682.
INFO:root:Creating index. Elapsed time since start: 0:00:20.233749.


<Response [200]>


INFO:root:Creating index. Elapsed time since start: 0:00:40.361452.


<Response [200]>


INFO:root:Creating index. Elapsed time since start: 0:01:00.465307.


...

<Response [200]>


INFO:root:Creating index. Elapsed time since start: 0:13:24.297172.


<Response [200]>


INFO:root:Operation completed!


<Response [200]>
{'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.Index', 'name': 'projects/895222332033/locations/us-central1/indexes/6981142372377690112'}


#### Verify that the index was created

In [45]:
indexes = index_client.list_indexes(index_display_name)

for index in indexes:
    print(index['name'])

if indexes: 
    index_id = index['name'].split('/')[-1]
    print(f'Index: {index_id} will be used for deployment')
else:
    print('No indexes available for deployment')

projects/895222332033/locations/us-central1/indexes/6981142372377690112
Index: 6981142372377690112 will be used for deployment


### Create the index deployment

#### Create an index endpoint

In [39]:
deployment_display_name = 'Song embeddings endpoint'

In [40]:
operation_id = deployment_client.create_endpoint(deployment_display_name, VPC_NAME)

response = index_client.wait_for_completion(operation_id, 'Waiting for endpoint', 10)
print(response)

INFO:root:Waiting for endpoint. Elapsed time since start: 0:00:00.130802.
INFO:root:Waiting for endpoint. Elapsed time since start: 0:00:10.223883.
INFO:root:Operation completed!


{'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.IndexEndpoint', 'name': 'projects/895222332033/locations/us-central1/indexEndpoints/8195425421907460096'}


#### Verify that the endpoint was created

In [46]:
endpoints = deployment_client.list_endpoints(deployment_display_name)

for endpoint in endpoints:
    print(endpoint['name'])
    
if endpoints: 
    endpoint_id = endpoint['name'].split('/')[-1]
    print(f'Endpoint: {endpoint_id} will be used for deployment')
else:
    print('No endpoints available for deployment')


projects/895222332033/locations/us-central1/indexEndpoints/8195425421907460096
Endpoint: 8195425421907460096 will be used for deployment


#### Deploy the index to the endpoint

##### Set the deployment ID

The ID must be unique within your project

In [47]:
deployment_display_name = 'Song embeddings deployed index'
deployed_index_id = 'songs_embeddings_deployed'

##### Deploy the index

Be patient. Index deployment is a long running operation

In [48]:
operation_id = deployment_client.create_deployment(deployment_display_name, 
                                                   deployed_index_id,
                                                   endpoint_id,
                                                   index_id)

response = index_client.wait_for_completion(operation_id, 'Waiting for deployment', 10)
print(response)

INFO:root:Waiting for deployment. Elapsed time since start: 0:00:00.100609.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:10.196557.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:20.284240.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:30.396930.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:40.466292.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:50.531255.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:00.623174.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:10.765113.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:20.851175.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:30.978102.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:41.061788.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:51.172531.
INFO:root:Waiting for deployment. Elapsed time since start: 0:02:01.266397.
INFO:root:Wa

{'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexResponse', 'deployedIndex': {'id': 'songs_embeddings_deployed'}}


#### Retrieve the gRPC private endpoint for the ANN Match service

In [49]:
deployed_index_ip = deployment_client.get_deployment_grpc_ip(endpoint_id, deployed_index_id)
endpoint = f'{deployed_index_ip}:{MATCH_SERVICE_PORT}'
print(f'gRPC endpoint for the: {deployed_index_id} deployment is: {endpoint}')

gRPC endpoint for the: songs_embeddings_deployed deployment is: 10.71.0.8:10000


## Querying the ANN service

You will use the gRPC interface to query the deployed index.

### Create a helper wrapper around the Match Service gRPC API.

The wrapper uses the pre-generated gRPC stub to the Match Service gRPC interface. 

In [50]:
class MatchService(object):
    def __init__(self, endpoint, deployed_index_id):
        self.endpoint = endpoint
        self.deployed_index_id = deployed_index_id

    def single_match(
        self,
        embedding: List[float],
        num_neighbors: int) -> List[Tuple[str, float]]:
    
        match_request = match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                               float_val=embedding,
                                               num_neighbors=num_neighbors)
        with grpc.insecure_channel(endpoint) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.Match(match_request)
    
        return [(neighbor.id, neighbor.distance) for neighbor in response.neighbor]


    def batch_match(
        self,
        embeddings: List[List[float]],
        num_neighbors: int) -> List[List[Tuple[str, float]]]:
    
        match_requests = [
            match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                   float_val=embedding,
                                   num_neighbors=num_neighbors)
            for embedding in embeddings]
    
        batches_per_index = [
            match_pb2.BatchMatchRequest.BatchMatchRequestPerIndex(
                deployed_index_id=self.deployed_index_id,
                requests=match_requests)]
    
        batch_match_request = match_pb2.BatchMatchRequest(
            requests=batches_per_index)
    
        with grpc.insecure_channel(endpoint) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.BatchMatch(batch_match_request)
        
        matches = []
        for batch_per_index in response.responses:
            for match in batch_per_index.responses:
                matches.append(
                    [(neighbor.id, neighbor.distance) for neighbor in match.neighbor])
        
        return matches
    
match_service = MatchService(endpoint, deployed_index_id)

### Prepare sample data

Retrieve a few embeddings from the BigQuery embedding table

In [51]:
%%bigquery df_embeddings

SELECT id, embedding
FROM `recommendations.item_embeddings` 
LIMIT 10

In [52]:
df_embeddings

,id,embedding
0,3106816,"[4.931899197016707, -13.04058337155901, -16.40..."
1,38144,"[-7.731296902716677, 32.163779115956835, -8.11..."
2,1583106,"[-16.862011403818066, 8.049872462652369, -12.9..."
3,1143298,"[10.865034961228991, -3.039872306630514, -0.55..."
4,2244100,"[-10.571954503767223, 7.44613508975598, 20.913..."
5,953605,"[-0.05320979327823455, -8.933988823490601, -3...."
6,3138565,"[-14.34828023904765, 16.482326503963087, -4.15..."
7,2174726,"[-8.715900308214637, 1.9351848653994654, 2.012..."
8,1126662,"[-12.513885938010324, 13.092930864130656, 1.67..."
9,3088647,"[-26.914009623655673, 2.8766353315928637, -9.0..."


### Run a single match query

The following call sends a single embeddings and requests 10 closest matches.

In [55]:
sample_embeddings = [list(embedding) for embedding in df_embeddings.iloc[0:2]['embedding']]

single_match = match_service.single_match(sample_embeddings[0], 10)
single_match

[('3212155', 0.5673990845680237),
 ('2113161', 0.564233124256134),
 ('564149', 0.5616567730903625),
 ('237408', 0.547609269618988),
 ('3399449', 0.5462256073951721),
 ('212252', 0.5443934202194214),
 ('3124218', 0.5440597534179688),
 ('105033', 0.5350871086120605),
 ('37153', 0.5348769426345825),
 ('119348', 0.5347628593444824)]

### Run a batch match query

The following call sends a batch of embeddings and requests 5 closest matches per embedding.

In [57]:
batch_match = match_service.batch_match(sample_embeddings, 5)
batch_match

[[('3212155', 0.5673990845680237),
  ('2113161', 0.564233124256134),
  ('564149', 0.5616567730903625),
  ('237408', 0.547609269618988),
  ('3399449', 0.5462256073951721)],
 [('928826', 0.4685501754283905),
  ('928799', 0.4668276309967041),
  ('3764633', 0.4653089940547943),
  ('1112280', 0.4528423845767975),
  ('2677020', 0.43297818303108215)]]

## Clean up

**WARNING**

The below code will delete all ANN deployments, endpoints, and indexes in the configured project.

### Delete index deployments and endpoints

In [25]:
for endpoint in deployment_client.list_endpoints():
    endpoint_id = endpoint['name'].split('/')[-1]
    if 'deployedIndexes' in endpoint.keys():
        for deployment in endpoint['deployedIndexes']:
            print('   Deleting index deployment: {} in the endpoint: {} '.format(deployment['id'], endpoint_id))
            deployment_client.delete_deployment(endpoint_id, deployment['id'])
    print('Deleting endpoint: {}'.format(endpoint['name']))
    deployment_client.delete_endpoint(endpoint_id)

Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/730709039540862976
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/1280148194080063488
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/8337288810169630720
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/3725602791742242816
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/8621015586693971968
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/3270739229377822720


### Delete indexes

In [28]:
for index in index_client.list_indexes():
    index_id = index['name'].split('/')[-1]
    print('Deleting index: {}'.format(index['name']))
    index_client.delete_index(index_id)

Deleting index: projects/895222332033/locations/us-central1/indexes/1246371196874784768
Deleting index: projects/895222332033/locations/us-central1/indexes/1444529580479086592
Deleting index: projects/895222332033/locations/us-central1/indexes/1160802803954745344


## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**